##### pysource

In [ ]:
#https://pysource.com/2021/10/29/kalman-filter-predict-the-trajectory-of-an-object/
import cv2
import numpy as np


class KalmanFilter:
    kf = cv2.KalmanFilter(4, 2)
    kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
    kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)


    def predict(self, coordX, coordY):
        ''' This function estimates the position of the object'''
        measured = np.array([[np.float32(coordX)], [np.float32(coordY)]])
        self.kf.correct(measured)
        predicted = self.kf.predict()
        x, y = int(predicted[0]), int(predicted[1])
        return x, y



In [ ]:
#https://pysource.com/2021/10/29/kalman-filter-predict-the-trajectory-of-an-object/
import cv2
import numpy as np


class OrangeDetector:
    def __init__(self):
        # Create mask for orange color
        self.low_orange = np.array([11, 128, 90])
        self.high_orange = np.array([179, 255, 255])

    def detect(self, frame):
        hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Create masks with color ranges
        mask = cv2.inRange(hsv_img, self.low_orange, self.high_orange)

        # Find Contours
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)

        box = (0, 0, 0, 0)
        for cnt in contours:
            (x, y, w, h) = cv2.boundingRect(cnt)
            box = (x, y, x + w, y + h)
            break

        return box

In [ ]:
#https://pysource.com/2021/10/29/kalman-filter-predict-the-trajectory-of-an-object/
import cv2
from orange_detector import OrangeDetector
from kalmanfilter import KalmanFilter

cap = cv2.VideoCapture(r"Pysource-Kalman-filter\Pysource Kalman filter\orange.mp4")

# Load detector
od = OrangeDetector()

# Load Kalman filter to predict the trajectory
kf = KalmanFilter()
print("d")
while True:
    print("True")
    ret, frame = cap.read()
    if ret is False:
        print("break")
        break

    orange_bbox = od.detect(frame)
    x, y, x2, y2 = orange_bbox
    cx = int((x + x2) / 2)
    cy = int((y + y2) / 2)

    predicted = kf.predict(cx, cy)
    #cv2.rectangle(frame, (x, y), (x2, y2), (255, 0, 0), 4)
    cv2.circle(frame, (cx, cy), 20, (0, 0, 255), 4)
    cv2.circle(frame, (predicted[0], predicted[1]), 20, (255, 0, 0), 4)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(150)
    if key == 27:
        break

##### GUI

In [ ]:
import cv2
import numpy as np

class MouseTracker:
    def __init__(self):
        self.kf = cv2.KalmanFilter(4, 2)
        self.kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
        self.kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)

        self.measured = False

    def update(self, x, y):
        if not self.measured:
            self.kf.statePost = np.array([[x], [y], [0], [0]], np.float32)
            self.measured = True

        predicted = self.kf.predict()
        measured = np.array([[np.float32(x)], [np.float32(y)]])
        corrected = self.kf.correct(measured)
        return int(corrected[0]), int(corrected[1])

def on_trackbar(val):
    pass

def main():
    tracker = MouseTracker()

    window_name = "Mouse Tracker"
    cv2.namedWindow(window_name)

    initial_x = 320
    initial_y = 240
    max_x = 640
    max_y = 480

    cv2.createTrackbar('X', window_name, initial_x, max_x, on_trackbar)
    cv2.createTrackbar('Y', window_name, initial_y, max_y, on_trackbar)

    while True:
        x = cv2.getTrackbarPos('X', window_name)
        y = cv2.getTrackbarPos('Y', window_name)

        predicted_x, predicted_y = tracker.update(x, y)

        frame = np.zeros((max_y, max_x, 3), dtype=np.uint8)
        cv2.circle(frame, (x, y), 10, (0, 255, 0), -1)
        cv2.circle(frame, (predicted_x, predicted_y), 10, (255, 0, 0), -1)

        cv2.imshow(window_name, frame)
        key = cv2.waitKey(10)
        if key == 27:
            break

    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


##### Chat GPT

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# カルマンフィルタの初期設定
def initialize_kalman_filter(initial_state, initial_covariance, process_variance, measurement_variance):
    state = initial_state
    covariance = initial_covariance
    transition_matrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]])
    observation_matrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]])
    process_noise_covariance = np.eye(4) * process_variance
    measurement_noise_covariance = np.eye(2) * measurement_variance

    return state, covariance, transition_matrix, observation_matrix, process_noise_covariance, measurement_noise_covariance

# カルマンフィルタの更新
def update_kalman_filter(state, covariance, transition_matrix, observation_matrix,
                         process_noise_covariance, measurement_noise_covariance, measurement):
    # 予測ステップ
    predicted_state = np.dot(transition_matrix, state)
    predicted_covariance = np.dot(np.dot(transition_matrix, covariance), transition_matrix.T) + process_noise_covariance

    # 更新ステップ
    innovation = measurement - np.dot(observation_matrix, predicted_state)
    innovation_covariance = np.dot(np.dot(observation_matrix, predicted_covariance), observation_matrix.T) + measurement_noise_covariance
    kalman_gain = np.dot(np.dot(predicted_covariance, observation_matrix.T), np.linalg.inv(innovation_covariance))
    
    updated_state = predicted_state + np.dot(kalman_gain, innovation)
    updated_covariance = np.dot(np.eye(4) - np.dot(kalman_gain, observation_matrix), predicted_covariance)
    
    return updated_state, updated_covariance

# シミュレーション用の真の状態と観測値を生成
np.random.seed(0)
true_state = np.array([5.0, 5.0, 1.0, 0.0])  # x, y, vx, vy
measurement_noise = 0.5
num_steps = 50
true_states = [true_state]
measurements = [true_state[:2] + np.random.normal(scale=measurement_noise, size=2)]

for _ in range(num_steps - 1):
    true_state[2:] += np.random.normal(scale=0.1, size=2)  # 速度のランダム性を追加
    true_state[:2] += true_state[2:]
    measurement = true_state[:2] + np.random.normal(scale=measurement_noise, size=2)
    true_states.append(true_state)
    measurements.append(measurement)

# カルマンフィルタの初期化
initial_state_estimate = np.array([measurements[0][0], measurements[0][1], 0, 0])
initial_covariance_estimate = np.diag([1, 1, 1, 1])
process_variance = 0.1
measurement_variance = measurement_noise ** 2
state_estimate, covariance_estimate, transition_matrix, observation_matrix, process_noise_covariance, measurement_noise_covariance = \
    initialize_kalman_filter(initial_state_estimate, initial_covariance_estimate, process_variance, measurement_variance)

# カルマンフィルタの更新と推定結果の保存
filtered_states = [state_estimate[:2]]
for measurement in measurements[1:]:
    state_estimate, covariance_estimate = update_kalman_filter(state_estimate, covariance_estimate,
                                                               transition_matrix, observation_matrix,
                                                               process_noise_covariance, measurement_noise_covariance,
                                                               measurement)
    filtered_states.append(state_estimate[:2])

# 結果のプロット
plt.figure(figsize=(10, 6))
true_states = np.array(true_states)
print(true_states)
measurements = np.array(measurements)
filtered_states = np.array(filtered_states)
plt.plot(true_states[:, 0], true_states[:, 1], label='True States', linestyle='dashed')
# plt.scatter(measurements[:, 0], measurements[:, 1], label='Measurements', marker='o')
# plt.plot(filtered_states[:, 0], filtered_states[:, 1], label='Filtered States', marker='x')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.title('Object Tracking using Kalman Filter (2D)')
plt.legend()
plt.grid(True)
plt.show()


##### youtube

In [ ]:
import cv2
import numpy as np

class RedPointDetector:
    def __init__(self):
        self.low_red = np.array([160,169,192])
        self.high_red = np.array([179,255,255])

        def detect(self,frame):
            hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

            mask = cv2.inRange(hsv_img, self.low_red, self.high_red)

            contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)

            box = (-100, -100, 0, 0)
            for cnt in contours:
                (x,y,w,h) = cv2.boundingRect(cnt)
                if w*h > 300:
                    box = (x,y,x+w,y+h)
                    break
            return box

In [ ]:
import cv2
import numpy as np
from Red_Point_Detector import RedPointDetector

class KalmanFilter:
    kf = cv2.KalmanFilter(4,2)
    kf.measurementMatrix = np.array([[1,0,0,0], [0,1,0,0]], np.float32)
    kf.transitionMatrix = np.array([[1,0,1,0],[0,1,0,1],[0,0,1,0],[0,0,0,1]])

    def predict(self, coordX, coordY):
        measured = np.array([[np.float32(coordX)], [np.float32(coordY)]])
        self.kf.correct(measured)
        predicted = self.kf.predict()
        x, y = int(predicted[0]), int(predicted[1])
        return x, y
    
dispW = 640
dispH = 480
flip = 2

# camSet_USB = 
cap = cv2.VideoCapture(0)

od = RedPointDetector()

kf = KalmanFilter()

# outVid = cv2.VideoWriter()
while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame,1)
    if ret is False:
        break

    redpoint_bbox = od.detect(frame)
    x, y, x2, y2 = redpoint_bbox
    cx = int((x + x2) / 2)
    cy = int((y + y2) / 2)

    predictd = kf.predict(cx, cy)
    cv2.circle(frame, (cx, cy), 10, (0, 255, 255), -1)
    cv2.circle(frame, (predictd[0], predictd[1]), 10, (255, 0, 0), 4)
     
    cv2.imshow(frame)  
    # outVid.write(frame)
    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
#outVid.release()
cv2.destroyAllWindows()



##### don't know

In [ ]:
import cv2
import numpy as np

# カルマンフィルタの初期設定
def initialize_kalman_filter(initial_state):
    kalman = cv2.KalmanFilter(4, 2)
    kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
    kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
    kalman.statePre = np.array([initial_state[0], initial_state[1], 0, 0], np.float32)
    kalman.statePost = np.array([initial_state[0], initial_state[1], 0, 0], np.float32)
    return kalman

# 動画読み込み
cap = cv2.VideoCapture('input_video.mp4')

# 初期物体位置
initial_position = np.array([100, 100], np.float32)
kalman_filter = initialize_kalman_filter(initial_position)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    measurement = np.array([[frame.shape[1] // 2], [frame.shape[0] // 2]], np.float32)
    
    # カルマンフィルタの更新
    prediction = kalman_filter.predict()
    corrected = kalman_filter.correct(measurement)
    filtered_position = (corrected[0], corrected[1])
    
    # 物体位置を描画
    cv2.circle(frame, (int(filtered_position[0]), int(filtered_position[1])), 10, (0, 255, 0), -1)
    
    cv2.imshow('Tracking', frame)
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
